### Load the model

In [1]:
import tensorflow as tf
import numpy
import h5py
from qkeras.utils import _add_supported_quantized_objects
from qkeras import quantized_bits
co = {}; _add_supported_quantized_objects(co)

model_path = f"/eos/project/c/cms-l1ml/public/Pawel/axol1tl_model.h5"

model = tf.keras.models.load_model(model_path, custom_objects=co)
#model_config = model.get_config()

2025-08-20 17:43:47.905117: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-20 17:43:47.960080: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-20 17:43:47.960133: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-20 17:43:47.960193: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-20 17:43:47.970643: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-20 17:43:47.971153: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
data_path = f"/eos/project/c/cms-l1ml/public/Pawel/complete.h5"
data_file = h5py.File(data_path, 'r')

### Copy weights into the model

In [3]:
#model.compile()

ds_weights = data_file['model']['trimmed_encoder']['model_weights']
for layer in iter(model.layers):
    #print(layer.name)
    if layer.name in ('input_1','subtract','dot'): continue

    if layer.name == 'q_dense_4':
        weights_group = ds_weights['q_dense_4']['variational_auto_encoder']['vae__encoder']['q_dense_4']
        layer.set_weights((weights_group['kernel:0'], weights_group['bias:0']))
        continue

    weights_group = ds_weights[layer.name][layer.name]
    layer.set_weights((weights_group['kernel:0'], weights_group['bias:0']))

### Prepare data for tests

In [4]:
input_quantizer = quantized_bits(14, 5, alpha=1)
output_quantizer = quantized_bits(18,13,alpha=1)
bias_quantizer = quantized_bits(18,12,alpha=1)

In [5]:
X = data_file['data']["Background_data"]["Test"]["DATA"][:2048]

X_flat = X.reshape(X.shape[0], -1)
X_flat = input_quantizer(X_flat)
X_flat.dtype

tf.float32

### Fragmentary model

In [11]:
fragm_model = tf.keras.Model(inputs=model.layers[0].input, outputs=model.layers[1].output, name='fragm_model')
fragm_model.summary()

Model: "fragm_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 57)]              0         
                                                                 
 q_dense (QDense)            (None, 29)                1682      
                                                                 
Total params: 1682 (6.57 KB)
Trainable params: 1682 (6.57 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Conversion into a hls4ml model

In [12]:
import hls4ml

hls_config = hls4ml.utils.config_from_keras_model(
    fragm_model,
    granularity='name', 
    default_precision='fixed<14,7>',
    backend='Vitis')
print("-----------------------------------")


Interpreting Model
Topology:
Layer name: input_1, layer type: InputLayer, input shapes: [[None, 57]], output shape: [None, 57]
Layer name: q_dense, layer type: QDense, input shapes: [[None, 57]], output shape: [None, 29]
-----------------------------------


/afs/cern.ch/user/p/psajdak/eos_mine/project_CERN/venv3_9/lib64/python3.9/site-packages/keras/src/constraints.py:365: UserWarning: The `keras.constraints.serialize()` API should only be used for objects of type `keras.constraints.Constraint`. Found an instance of type <class 'qkeras.quantizers.quantized_bits'>, which may lead to improper serialization.
  warnings.warn(


In [13]:
hls_model = hls4ml.converters.convert_from_keras_model(
    fragm_model, hls_config=hls_config, backend='Vitis',
    output_dir='./hls_fragm_0_1'
)

Interpreting Model
Topology:
Layer name: input_1, layer type: InputLayer, input shapes: [[None, 57]], output shape: [None, 57]
Layer name: q_dense, layer type: QDense, input shapes: [[None, 57]], output shape: [None, 29]
Creating HLS model


In [14]:
hls_model.compile()

Writing HLS project


Done


### Compare predictions for the QKeras and HLS model

In [ ]:
pred_qkeras = fragm_model.predict(X_flat)
pred_hls = hls_model.predict(numpy.ascontiguousarray(X_flat))

64/64 [==============================] - 0s 2ms/step


Both shapes (2048,29) and dtypes (float32) consistent

In [23]:
numpy.vstack((pred_qkeras[0], pred_hls[0]))

array([[1.625 , 0.625 , 0.75  , 1.1875, 0.5   , 0.75  , 1.5625, 3.6875,
        1.0625, 1.625 , 0.    , 0.    , 0.4375, 2.3125, 1.0625, 0.    ,
        3.6875, 0.    , 0.    , 2.125 , 2.875 , 0.    , 0.    , 1.25  ,
        0.    , 0.125 , 0.125 , 3.875 , 0.    ],
       [1.5625, 0.5625, 0.6875, 1.125 , 0.4375, 0.6875, 1.5   , 3.6875,
        1.    , 1.5625, 0.    , 0.    , 0.375 , 2.25  , 1.0625, 0.    ,
        3.625 , 0.    , 0.    , 2.0625, 2.8125, 0.    , 0.    , 1.1875,
        0.    , 0.0625, 0.0625, 3.8125, 0.    ]], dtype=float32)